In [10]:
import pandas as pd
import datetime as dt
import os

In [11]:
#final code_block <- works

def mod_concat_NEMDE(file_path):
    df = pd.read_pickle(file_path)
    df.drop(columns=["BandNo","RRNBandPrice","BandCost"], inplace=True)
    df = df.set_index('PeriodID', drop=True)
    df.index = pd.to_datetime(df.index).tz_localize(None)
    df[["Price", "Increase"]] = df[["Price", "Increase"]].astype(float)
    df.sort_index(inplace=True, ascending=False)
    df.index = df.index - dt.timedelta(minutes = 5)
    df = df.loc[df.index < dt.datetime(2018,8,1), :]
    assert df["Price"].dtype == float
    assert df["Increase"].dtype == float
    assert df.isna().any().any() == False
    return df

In [12]:
df = mod_concat_NEMDE('{}/nem-data/nemde_clean/concat_for_energy_SA1.pkl'.format(os.environ['HOME']))
df.head()

,RegionID,Market,Price,Unit,DispatchedMarket,Increase
PeriodID,,,,,,
2018-07-31 23:55:00,SA1,Energy,70.70204,ER01,ENOF,0.34712
2018-07-31 23:55:00,SA1,Energy,70.70204,ER04,ENOF,0.31058
2018-07-31 23:55:00,SA1,Energy,70.70204,ER03,ENOF,0.34712
2018-07-31 23:55:00,SA1,Energy,70.70204,ER02,ENOF,0.31058
2018-07-31 23:50:00,SA1,Energy,69.24109,DEVILS_G,L5MI,-1.29132


In [16]:
df = df.groupby(df.index).mean().sort_index(ascending=False)
df.head()

,Price,Increase
PeriodID,,
2018-07-31 23:55:00,70.70204,0.328850
2018-07-31 23:50:00,69.24109,48.926729
2018-07-31 23:45:00,69.94850,0.325345
2018-07-31 23:40:00,74.04497,0.638320
2018-07-31 23:35:00,76.44245,9.705137


In [17]:
def correct_timedelta(df, time_diff):
    '''
    df.index must be DateTimeIndex
    Returns two lists
    df=table_of_interest
    col="column_of_interest"
    time_diff=time_diff in seconds as int
    '''
    lst = []
    lst_i = []
    count = 0
    for i in df.index:
        count += 1
        if count >= len(df):
            break
        delta = abs(df.index[count] - df.index[count-1])
        if int(delta.total_seconds()) != int(time_diff):
            lst.append(("from index {} on, it has been {} s or {} h.".format(count-1,int(delta.total_seconds()),(int(delta.total_seconds()/3600)))))
            lst_i.append((df.index[count-1],int(delta.total_seconds())))
    return lst, lst_i

In [18]:
lst, lst_i = correct_timedelta(df, 300)

In [19]:
lst_i

[(Timestamp('2018-07-01 04:00:00'), 2592300),
 (Timestamp('2018-04-10 13:30:00'), 600),
 (Timestamp('2018-03-24 04:00:00'), 600),
 (Timestamp('2017-10-11 23:00:00'), 600),
 (Timestamp('2017-10-07 22:00:00'), 600),
 (Timestamp('2016-11-07 15:35:00'), 600),
 (Timestamp('2016-10-15 14:35:00'), 600),
 (Timestamp('2016-10-15 13:30:00'), 600),
 (Timestamp('2016-10-15 11:55:00'), 600),
 (Timestamp('2016-10-15 11:35:00'), 600),
 (Timestamp('2016-10-15 11:00:00'), 600),
 (Timestamp('2016-10-15 09:55:00'), 600),
 (Timestamp('2016-10-15 09:35:00'), 1200),
 (Timestamp('2016-10-15 07:45:00'), 600),
 (Timestamp('2016-09-09 03:20:00'), 600),
 (Timestamp('2016-09-09 02:55:00'), 600),
 (Timestamp('2016-09-08 23:00:00'), 600),
 (Timestamp('2016-08-19 23:40:00'), 600),
 (Timestamp('2016-08-18 23:20:00'), 900),
 (Timestamp('2016-08-18 23:05:00'), 600),
 (Timestamp('2016-08-16 02:20:00'), 600),
 (Timestamp('2016-08-16 02:00:00'), 600),
 (Timestamp('2016-08-15 23:05:00'), 600),
 (Timestamp('2016-08-01 23:40

In [20]:
len(lst_i)

44

In [8]:
df1 = mod_concat_NEM_test('{}/nem-data/nemde_clean/chunks_clean/1.pkl'.format(os.environ['HOME']))

In [9]:
print(df1.index.min())
print(df1.index.max())
df1.head()

2009-07-01 04:00:00
2010-05-01 03:55:00


,RegionID,Market,Price,Unit,DispatchedMarket,BandNo,Increase,RRNBandPrice,BandCost
PeriodID,,,,,,,,,
2010-05-01 03:55:00,TAS1,L60S,0.10000,BASTYAN,L60S,2,-1.00000,0.10000,-0.10000
2010-05-01 03:55:00,NSW1,L5RE,2.00000,CETHANA,L5RE,3,-1.00000,2.00000,-2.00000
2010-05-01 03:55:00,SA1,Energy,17.20054,BASTYAN,L60S,2,0.95665,0.10000,0.09566
2010-05-01 03:55:00,SA1,Energy,17.20054,JBUTTERS,ENOF,3,0.95665,18.28000,17.48753
2010-05-01 03:55:00,SA1,Energy,17.20054,LD01,L60S,2,-0.95665,0.40000,-0.38266


In [10]:
df2 = mod_concat_NEM_test('{}/nem-data/nemde_clean/chunks_clean/2.pkl'.format(os.environ['HOME']))

In [11]:
print(df2.index.min())
print(df2.index.max())
df2.head()

2010-05-01 04:00:00
2011-03-01 03:55:00


,RegionID,Market,Price,Unit,DispatchedMarket,BandNo,Increase,RRNBandPrice,BandCost
PeriodID,,,,,,,,,
2011-03-01 03:55:00,TAS1,R6SE,1.10000,GORDON,R6SE,4,-1.00000,1.10000,-1.10000
2011-03-01 03:55:00,QLD1,Energy,16.75000,TARONG#2,ENOF,3,0.50000,16.75000,8.37500
2011-03-01 03:55:00,SA1,Energy,14.31085,TARONG#1,ENOF,3,0.42719,16.75000,7.15543
2011-03-01 03:55:00,QLD1,L5RE,2.42000,LYA3,L5RE,3,-1.00000,2.42000,-2.42000
2011-03-01 03:55:00,QLD1,L5MI,1.35000,TARONG#2,L5MI,3,-1.00000,1.35000,-1.35000
